# Web scraping Tutorial

* This notebook is a quick reference on how to use beautifulsoup4 to download data from websites- specifically basketball-reference.com

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import os

In [2]:
url = 'https://www.basketball-reference.com/players/e/embiijo01/gamelog/2021'
page = requests.get(url)
page

<Response [200]>

In [3]:
html = urlopen(url)
html

In [4]:
soup = BeautifulSoup(html)
#print(soup.prettify())

In [5]:
# using findAll toget the column headers

soup.findAll('thead')

headers = [th.getText() for th in soup.findAll('thead')[0].findAll('th')]
headers 

['Rk',
 'G',
 'Date',
 'Age',
 'Tm',
 '\xa0',
 'Opp',
 '\xa0',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [10]:
headers.remove('Rk')
#remove extra header
headers

['G',
 'Date',
 'Age',
 'Tm',
 '\xa0',
 'Opp',
 '\xa0',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'GmSc',
 '+/-']

In [11]:
#stats = [[td.gettext() for td in soup.findAll('td', {'data-stat': stat})] for stat in headers]
# shrink it down to just the data table stats

# this table_body returns as a list
#table_body = soup.findAll('tbody')
#table_body

#def table_body(soup):
   # return soup.has_attr('data-stat')
#rows = table_body.find_all('tr')

table_body = soup.tbody
#table_body

rows = table_body.findAll('tr')
#rows


In [12]:
#rk = soup.find_all(attrs={'data-stat':'ranker'})
#rk


In [13]:
player_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
#player_stats

In [14]:
gamelog = pd.DataFrame(player_stats, columns = headers)
#gamelog

In [15]:
gamelog.to_csv("joelembiid_gamelog.csv")

In [16]:
df_embiid = pd.read_csv('joelembiid_gamelog.csv')

In [17]:
df_embiid["FPTS"] = (df_embiid.PTS) + (df_embiid.TRB*1.2) + (df_embiid.AST*1.5) + (df_embiid.STL*3) + (df_embiid.BLK*3) + (df_embiid.TOV*-1)

In [29]:
# fucntion to turn minutes played into a float. 

def min_to_float(x):
    if pd.isna(x)==True:
        return 0
    else:
        mins = int(x.split(':')[0])
        secs = int(x.split(':')[1])/60.0
        val = mins + secs
        return round(val, 1)

# apply to df

df_embiid['MP'] = df_embiid['MP'].apply(min_to_float)

In [30]:
# binary for game played vs not played
df_embiid['g'] = [1 if pd.isna(df_embiid.G[gm])==False else 0 for gm in range(len(df_embiid))]
df_embiid.head()

,Unnamed: 0,G,Date,Age,Tm,Opp,outcome,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,FPTS,g,HOME_GAME,back_to_back,win
0,0,1.0,2020-12-23,26-282,PHI,WAS,W (+6),1,34.8,10.0,17.0,0.588,1.0,1.0,1.000,8.0,9.0,0.889,2.0,12.0,14.0,2.0,0.0,1.0,3.0,3.0,29.0,23.6,-2.0,48.8,1,1,0,1
1,1,2.0,2020-12-26,26-285,PHI,NYK,W (+20),1,30.6,10.0,20.0,0.500,2.0,4.0,0.500,5.0,7.0,0.714,4.0,6.0,10.0,2.0,1.0,1.0,1.0,3.0,27.0,21.7,18.0,47.0,1,0,0,1
2,2,NaN,2020-12-27,26-286,PHI,CLE,L (-24),Did Not Dress,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0
3,3,3.0,2020-12-29,26-288,PHI,TOR,W (+7),1,37.9,7.0,17.0,0.412,1.0,6.0,0.167,14.0,16.0,0.875,4.0,12.0,16.0,4.0,2.0,2.0,5.0,1.0,29.0,26.3,15.0,61.2,1,1,0,1
4,4,4.0,2020-12-31,26-290,PHI,ORL,W (+24),1,26.7,7.0,11.0,0.636,1.0,2.0,0.500,6.0,8.0,0.750,0.0,9.0,9.0,2.0,0.0,2.0,0.0,2.0,21.0,20.0,23.0,40.8,1,0,0,1


In [31]:
# change column name
column_names = {'': 'game_loc', '.1': 'outcome'}
df_embiid.columns = df_embiid.columns.str.strip()
df_embiid = df_embiid.rename(columns = column_names)
df_embiid.columns

Index(['Unnamed: 0', 'G', 'Date', 'Age', 'Tm', 'Opp', 'outcome', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-',
       'FPTS', 'g', 'HOME_GAME', 'back_to_back', 'win'],
      dtype='object')

In [32]:
df_embiid['HOME_GAME'] = [1 if pd.isna(df_embiid.game_loc[gm])==True else 0 for gm in range(len(df_embiid))]
df_embiid.drop(['game_loc'], inplace=True, axis=1)
df_embiid.head()

AttributeError: ignored

In [33]:
# add back-to-back indicator
# df_embiid['back_to_back'] = [1 if ]
df_embiid['Date'] = pd.to_datetime(df_embiid['Date'])

# making column out of the difference in days. 
df_embiid['back_to_back'] = pd.to_numeric(df_embiid.Date.dt.day.diff(), downcast='integer', errors='coerce')


# need to make a column where 1 for back to back and zero for not
df_embiid['back_to_back'] = [1 if (df_embiid.back_to_back[x] == 1.0) == True else 0 for x in range(len(df_embiid))]




df_embiid.head()

,Unnamed: 0,G,Date,Age,Tm,Opp,outcome,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,FPTS,g,HOME_GAME,back_to_back,win
0,0,1.0,2020-12-23,26-282,PHI,WAS,W (+6),1,34.8,10.0,17.0,0.588,1.0,1.0,1.000,8.0,9.0,0.889,2.0,12.0,14.0,2.0,0.0,1.0,3.0,3.0,29.0,23.6,-2.0,48.8,1,1,0,1
1,1,2.0,2020-12-26,26-285,PHI,NYK,W (+20),1,30.6,10.0,20.0,0.500,2.0,4.0,0.500,5.0,7.0,0.714,4.0,6.0,10.0,2.0,1.0,1.0,1.0,3.0,27.0,21.7,18.0,47.0,1,0,0,1
2,2,NaN,2020-12-27,26-286,PHI,CLE,L (-24),Did Not Dress,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0
3,3,3.0,2020-12-29,26-288,PHI,TOR,W (+7),1,37.9,7.0,17.0,0.412,1.0,6.0,0.167,14.0,16.0,0.875,4.0,12.0,16.0,4.0,2.0,2.0,5.0,1.0,29.0,26.3,15.0,61.2,1,1,0,1
4,4,4.0,2020-12-31,26-290,PHI,ORL,W (+24),1,26.7,7.0,11.0,0.636,1.0,2.0,0.500,6.0,8.0,0.750,0.0,9.0,9.0,2.0,0.0,2.0,0.0,2.0,21.0,20.0,23.0,40.8,1,0,0,1


In [34]:
# getting wins into binary
df_embiid['win'] = [1 if df_embiid['outcome'][gm][:1] == 'W' else 0 for gm in range(len(df_embiid))]
sum(df_embiid.win)

12

In [35]:
# would like to change 'outcome' to just a positive or negative integer


In [36]:
# function to get normalized stats
def stat_per_time(stat, minutes, per=36):
  if minutes == 0:
    return 0
  else:
    val_min = stat/minutes
    return round(val_min * per, 1)

# I want to see the normalized stat for fantasy points. but this can be used for any column. 

df_embiid['fpts_per_36'] = df_embiid.apply(lambda row: stat_per_time(stat = row['FPTS'], minutes = row['MP']), axis=1)


In [37]:
df_embiid

,Unnamed: 0,G,Date,Age,Tm,Opp,outcome,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,FPTS,g,HOME_GAME,back_to_back,win,fpts_per_36
0,0,1.0,2020-12-23,26-282,PHI,WAS,W (+6),1,34.8,10.0,17.0,0.588,1.0,1.0,1.000,8.0,9.0,0.889,2.0,12.0,14.0,2.0,0.0,1.0,3.0,3.0,29.0,23.6,-2.0,48.8,1,1,0,1,50.5
1,1,2.0,2020-12-26,26-285,PHI,NYK,W (+20),1,30.6,10.0,20.0,0.500,2.0,4.0,0.500,5.0,7.0,0.714,4.0,6.0,10.0,2.0,1.0,1.0,1.0,3.0,27.0,21.7,18.0,47.0,1,0,0,1,55.3
2,2,NaN,2020-12-27,26-286,PHI,CLE,L (-24),Did Not Dress,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0.0
3,3,3.0,2020-12-29,26-288,PHI,TOR,W (+7),1,37.9,7.0,17.0,0.412,1.0,6.0,0.167,14.0,16.0,0.875,4.0,12.0,16.0,4.0,2.0,2.0,5.0,1.0,29.0,26.3,15.0,61.2,1,1,0,1,58.1
4,4,4.0,2020-12-31,26-290,PHI,ORL,W (+24),1,26.7,7.0,11.0,0.636,1.0,2.0,0.500,6.0,8.0,0.750,0.0,9.0,9.0,2.0,0.0,2.0,0.0,2.0,21.0,20.0,23.0,40.8,1,0,0,1,55.0
5,5,5.0,2021-01-02,26-292,PHI,CHO,W (+15),1,36.7,7.0,11.0,0.636,1.0,2.0,0.500,4.0,6.0,0.667,3.0,11.0,14.0,4.0,1.0,1.0,5.0,2.0,19.0,17.4,19.0,42.8,1,1,0,1,42.0
6,6,6.0,2021-01-04,26-294,PHI,CHO,W (+17),1,24.9,4.0,10.0,0.400,0.0,3.0,0.000,6.0,8.0,0.750,1.0,10.0,11.0,4.0,2.0,3.0,1.0,1.0,14.0,17.0,18.0,47.2,1,1,0,1,68.2
7,7,7.0,2021-01-06,26-296,PHI,WAS,W (+5),1,36.6,11.0,20.0,0.550,3.0,4.0,0.750,13.0,13.0,1.000,0.0,8.0,8.0,5.0,3.0,3.0,5.0,1.0,38.0,34.0,14.0,68.1,1,1,0,1,67.0
8,8,8.0,2021-01-07,26-297,PHI,BRK,L (-13),1,29.9,7.0,14.0,0.500,2.0,2.0,1.000,4.0,5.0,0.800,0.0,12.0,12.0,3.0,0.0,1.0,5.0,5.0,20.0,12.0,-16.0,36.9,1,0,1,0,44.4
9,9,NaN,2021-01-09,26-299,PHI,DEN,L (-12),Inactive,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0.0


In [38]:
df_embiid.to_csv("joelembiid_gamelog.csv")